In [ ]:
import osmnx as ox
import matplotlib.pyplot as plt
import networkx as nx
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
import pickle
import heapq
import math
import folium
import pandas as pd
from collections import deque

Generating Graph for pedestrian pathway using OpenSourceMaps library osmnx

In [ ]:
# Set Location
place_name = "University of New Haven,CT,USA"

# Fetch the street network for the set location
G = ox.graph_from_place(place_name, network_type='walk')

# Plot the graph along with place name and geodata
fig, ax = ox.plot_graph(G, show=False, close=False)
ax.set_title(place_name, fontsize=12)  # Set the place name as the title

# Visualise
plt.show()

Above graph generated does not contain proper weights for each edge and node data does not contain name of the place it points.Calculating distance between two nodes of the edges in meters and reverse geocoding node name and address using it's lat and long co-ordinates.

Please insert your Google maps API key inplace of google_maps_api_key

In [ ]:
# Initialize a geolocator for reverse geocoding
geolocator = GoogleV3(api_key=google_maps_api_key)

# Calculate geographical distances between nodes and set them as edge weights
for u, v, data in G.edges(data=True):
    # Get the latitude and longitude of the nodes
    coords_u = (G.nodes[u]['y'], G.nodes[u]['x'])
    coords_v = (G.nodes[v]['y'], G.nodes[v]['x'])
    
    # Compute the geographical distance between nodes using geodesic
    distance = geodesic(coords_u, coords_v).meters  # Distance in meters
    
    # Set the edge attribute 'weight' with the computed distance
    G[u][v][0]['weight'] = distance  # Setting for the first (0th) edge attribute

# Reverse geocode node names to addresses and update node names accordingly
for node in G.nodes():
    # Get the latitude and longitude of the node
    lat = G.nodes[node]['y']
    lon = G.nodes[node]['x']
    # Reverse geocode to get the address
    location = geolocator.reverse((lat, lon), exactly_one=True)
    if location:
        name = location.address.split(',')[0]
        address=location.address
    
    # Update the node attribute 'address' with the reverse geocoded address
    G.nodes[node]['address'] = address

    # Update the node attribute 'name' with only the node name
    G.nodes[node]['name'] = name
    print("H",G.nodes[node])

Configuration for Graph Generated

In [ ]:
print("Number of Nodes",loaded_graph.number_of_nodes())
print("Number of Edges",loaded_graph.number_of_edges())

Saving Edge data

In [ ]:
edge_pd=[]
for start,end ,data in G.edges(data=True):
    data['start']=start
    data['end']=end
    edge_pd.append(data)

In [ ]:
#saving edge dataframe 
edge_pd=pd.DataFrame(edge_pd).to_csv('Ai_project_edge_data.csv')


In [ ]:
edge_pd.head(5)

In [ ]:
node_pd=[]
for x,data in G.nodes(data=True):
    data['id']=x
    node_pd.append(data)

In [ ]:
#saving node dataframe 
node_pd=pd.DataFrame(node_pd).to_csv('Ai_project_node_data.csv')


In [ ]:
node_pd

Visualizing Full Graph

In [ ]:
# Create a folium map centered at the first node
mymap = folium.Map(location=list((loaded_graph.nodes[178537304]['y'],loaded_graph.nodes[178537304]['x'])), zoom_start=50)

# Plot each node on the map with its name
for node, coordinates in loaded_graph.nodes(data=True):
#     print(node,(loaded_graph.nodes[node]['y'],loaded_graph.nodes[node]['x']),coordinates['name'],coordinates)
#     folium.Marker(location=(loaded_graph.nodes[node]['y'],loaded_graph.nodes[node]['x']), popup=node).add_to(mymap)
    folium.Marker(location=(loaded_graph.nodes[node]['y'],loaded_graph.nodes[node]['x']),popup=node).add_to(mymap)

# Plot edges on the map
for start,end,edge in loaded_graph.edges(data=True):
#     print(start,end,edge)
    start_node=start
    end_node=end
    if start_node==end_node or 'weight' not in edge:
        weight=0
    else:weight = edge['weight']
    folium.PolyLine([(loaded_graph.nodes[start_node]['y'],loaded_graph.nodes[start_node]['x']), (loaded_graph.nodes[end_node]['y'],loaded_graph.nodes[end_node]['x'])], color="black", weight=2.5, opacity=1, popup=f"{weight:.2f} meters").add_to(mymap)

# Save the map to an HTML file
mymap.save("map_with_nodes.html")

Set Start and End Nodes based on Id from node_pd

In [ ]:
start_test=2096427235
end_test=3437140637


Creating base code for A* Algorithm

In [ ]:
def aStarSearch(graph,start,end,heuristic):
    print("start",start)
    print("end",end)
    open_list = [(0, start)]
    heapq.heapify(open_list)
    
    parents = {}
    g_values = {node: float('inf') for node in graph.nodes()}
    g_values[start] = 0
    nodes_expanded=[start]
    count=0
    while open_list:
        
        
        _, current_node = heapq.heappop(open_list)
        count+=1
        nodes_expanded.append(current_node)
        if current_node == end:
            path = [end]
            while end != start:
                path.append(parents[end])
                end = parents[end]
            print("Node Visited",count)
            return (path[::-1],nodes_expanded)
        
        for neighbor in graph.neighbors(current_node):            
            tentative_g_value = g_values[current_node] + graph[current_node][neighbor][0]['weight']
            
            if tentative_g_value < g_values[neighbor]:
                parents[neighbor] = current_node
                g_values[neighbor] = tentative_g_value
                
                f_value = tentative_g_value + heuristic(graph.nodes[neighbor], graph.nodes[end])
                heapq.heappush(open_list, (f_value, neighbor))
                
    
    return None

Function to calculate Haversine Heuristic

In [ ]:
def haversine(neighbor,end):
#     print("neighbor",neighbor)
#     print("end",end)
    distance_meters = calculate_distance(neighbor['x'],neighbor['y'], end['x'], end['y'])
    return distance_meters

def calculate_distance(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)
    
    # Haversine formula to calculate distance between two points on Earth
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = 6371 * c * 1000  # Radius of Earth in kilometers * distance (in meters)
    
    return distance

Function to calculate Euclidean Distance Heuristic

In [ ]:
def euclidean_distance(node1, node2):
    x1, y1 = node1['x'], node1['y']
    x2, y2 = node2['x'], node2['y']
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

Function to calculate Octile Distance Heuristic

In [ ]:
def octile_distance(node1, node2):
    dx = abs(node2['x'] - node1['x'])
    dy = abs(node2['y'] - node1['y'])
    return max(dx, dy) + (math.sqrt(2) - 1) * min(dx, dy)

Function to calculate Chebyshev Distance Heuristic

In [ ]:
def chebyshev_distance(node1, node2):
    dx = abs(node2['x'] - node1['x'])
    dy = abs(node2['y'] - node1['y'])
    return max(dx, dy)

Function for Null Heuristic

In [ ]:
def null_heuristic(neighbor,end):
    return 0

A* with Null Heuristic

In [ ]:
#A* Search null
shortest_path,nodes_expanded=aStarSearch(loaded_graph,start_test,end_test,null_heuristic)

Plot A* path and nodes expanded with null heuristic

In [ ]:
#Plot A* Map
# Create a folium map centered at the first node
mymap = folium.Map(location=list((loaded_graph.nodes[178537304]['y'],loaded_graph.nodes[178537304]['x'])), zoom_start=50)

# Plot each node on the map with its name
for node, coordinates in loaded_graph.nodes(data=True):
#     print(node,(loaded_graph.nodes[node]['y'],loaded_graph.nodes[node]['x']),coordinates['name'],coordinates)
#     folium.Marker(location=(loaded_graph.nodes[node]['y'],loaded_graph.nodes[node]['x']), popup=node).add_to(mymap)
    folium.Marker(location=(loaded_graph.nodes[node]['y'],loaded_graph.nodes[node]['x']),popup=node).add_to(mymap)

# Plot edges on the map
for start,end,edge in loaded_graph.edges(data=True):
#     print(start,end,edge)
    start_node=start
    end_node=end
    if start_node==end_node or 'weight' not in edge:
        weight=0
    else:weight = edge['weight']
    folium.PolyLine([(loaded_graph.nodes[start_node]['y'],loaded_graph.nodes[start_node]['x']), (loaded_graph.nodes[end_node]['y'],loaded_graph.nodes[end_node]['x'])], color="black", weight=2.5, opacity=1, popup=f"{weight:.2f} meters").add_to(mymap)
# Plot the expanded nodes

for node in nodes_expanded:
    path_coordinates = [(loaded_graph.nodes[neighbour]['y'],loaded_graph.nodes[neighbour]['x'])for neighbour in loaded_graph.neighbors(node)]
    folium.PolyLine(path_coordinates, color="Yellow", weight=5, opacity=1).add_to(mymap)

# Plot the shortest path in red
distance=0
for i  in range(len(shortest_path)-1):
#     print(loaded_graph.get_edge_data(shortest_path[i],shortest_path[i+1])[0])
    distance+=loaded_graph.get_edge_data(shortest_path[i],shortest_path[i+1])[0]['weight']
path_coordinates = [(loaded_graph.nodes[node]['y'],loaded_graph.nodes[node]['x'])for node in shortest_path]
folium.PolyLine(path_coordinates, color="red", weight=5, opacity=1, popup=f"Shortest Path: {distance:.2f} meters").add_to(mymap)

# Save the map to an HTML file
mymap.save("map_with_nodes_Astar_null.html")

A* with Haversine Heuristic

In [ ]:
#A* Search haversine
shortest_path,nodes_expanded=aStarSearch(loaded_graph,start_test,end_test,haversine)

Plot A* path and nodes expanded with Haversine heuristic

In [ ]:
#Plot A* Map
# Create a folium map centered at the first node
mymap = folium.Map(location=list((loaded_graph.nodes[178537304]['y'],loaded_graph.nodes[178537304]['x'])), zoom_start=50)

# Plot each node on the map with its name
for node, coordinates in loaded_graph.nodes(data=True):
#     print(node,(loaded_graph.nodes[node]['y'],loaded_graph.nodes[node]['x']),coordinates['name'],coordinates)
#     folium.Marker(location=(loaded_graph.nodes[node]['y'],loaded_graph.nodes[node]['x']), popup=node).add_to(mymap)
    folium.Marker(location=(loaded_graph.nodes[node]['y'],loaded_graph.nodes[node]['x']),popup=node).add_to(mymap)

# Plot edges on the map
for start,end,edge in loaded_graph.edges(data=True):
#     print(start,end,edge)
    start_node=start
    end_node=end
    if start_node==end_node or 'weight' not in edge:
        weight=0
    else:weight = edge['weight']
    folium.PolyLine([(loaded_graph.nodes[start_node]['y'],loaded_graph.nodes[start_node]['x']), (loaded_graph.nodes[end_node]['y'],loaded_graph.nodes[end_node]['x'])], color="black", weight=2.5, opacity=1, popup=f"{weight:.2f} meters").add_to(mymap)
# Plot the expanded nodes

for node in nodes_expanded:
    path_coordinates = [(loaded_graph.nodes[neighbour]['y'],loaded_graph.nodes[neighbour]['x'])for neighbour in loaded_graph.neighbors(node)]
    folium.PolyLine(path_coordinates, color="Yellow", weight=5, opacity=1).add_to(mymap)

# Plot the shortest path in red
distance=0
for i  in range(len(shortest_path)-1):
#     print(loaded_graph.get_edge_data(shortest_path[i],shortest_path[i+1])[0])
    distance+=loaded_graph.get_edge_data(shortest_path[i],shortest_path[i+1])[0]['weight']
path_coordinates = [(loaded_graph.nodes[node]['y'],loaded_graph.nodes[node]['x'])for node in shortest_path]
folium.PolyLine(path_coordinates, color="red", weight=5, opacity=1, popup=f"Shortest Path: {distance:.2f} meters").add_to(mymap)

# Save the map to an HTML file
mymap.save("map_with_nodes_Astar_haversine.html")

A* with Eculidean Distance Heuristic

In [ ]:
#A* Search eculidean
shortest_path,nodes_expanded=aStarSearch(loaded_graph,start_test,end_test,euclidean_distance)

Plot A* path and nodes expanded with Eculidean Distance heuristic

In [ ]:
#Plot A* Map
# Create a folium map centered at the first node
mymap = folium.Map(location=list((loaded_graph.nodes[178537304]['y'],loaded_graph.nodes[178537304]['x'])), zoom_start=50)

# Plot each node on the map with its name
for node, coordinates in loaded_graph.nodes(data=True):
#     print(node,(loaded_graph.nodes[node]['y'],loaded_graph.nodes[node]['x']),coordinates['name'],coordinates)
#     folium.Marker(location=(loaded_graph.nodes[node]['y'],loaded_graph.nodes[node]['x']), popup=node).add_to(mymap)
    folium.Marker(location=(loaded_graph.nodes[node]['y'],loaded_graph.nodes[node]['x']),popup=node).add_to(mymap)

# Plot edges on the map
for start,end,edge in loaded_graph.edges(data=True):
#     print(start,end,edge)
    start_node=start
    end_node=end
    if start_node==end_node or 'weight' not in edge:
        weight=0
    else:weight = edge['weight']
    folium.PolyLine([(loaded_graph.nodes[start_node]['y'],loaded_graph.nodes[start_node]['x']), (loaded_graph.nodes[end_node]['y'],loaded_graph.nodes[end_node]['x'])], color="black", weight=2.5, opacity=1, popup=f"{weight:.2f} meters").add_to(mymap)
# Plot the expanded nodes

for node in nodes_expanded:
    path_coordinates = [(loaded_graph.nodes[neighbour]['y'],loaded_graph.nodes[neighbour]['x'])for neighbour in loaded_graph.neighbors(node)]
    folium.PolyLine(path_coordinates, color="Yellow", weight=5, opacity=1).add_to(mymap)

# Plot the shortest path in red
distance=0
for i  in range(len(shortest_path)-1):
#     print(loaded_graph.get_edge_data(shortest_path[i],shortest_path[i+1])[0])
    distance+=loaded_graph.get_edge_data(shortest_path[i],shortest_path[i+1])[0]['weight']
path_coordinates = [(loaded_graph.nodes[node]['y'],loaded_graph.nodes[node]['x'])for node in shortest_path]
folium.PolyLine(path_coordinates, color="red", weight=5, opacity=1, popup=f"Shortest Path: {distance:.2f} meters").add_to(mymap)

# Save the map to an HTML file
mymap.save("map_with_nodes_Astar_eculidean.html")

A* with Octile Distance Heuristic

In [ ]:
#A* Search octile
shortest_path,nodes_expanded=aStarSearch(loaded_graph,start_test,end_test,octile_distance)

Plot A* path and nodes expanded with Octile Distance heuristic

In [ ]:
#Plot A* Map
# Create a folium map centered at the first node
mymap = folium.Map(location=list((loaded_graph.nodes[178537304]['y'],loaded_graph.nodes[178537304]['x'])), zoom_start=50)

# Plot each node on the map with its name
for node, coordinates in loaded_graph.nodes(data=True):
#     print(node,(loaded_graph.nodes[node]['y'],loaded_graph.nodes[node]['x']),coordinates['name'],coordinates)
#     folium.Marker(location=(loaded_graph.nodes[node]['y'],loaded_graph.nodes[node]['x']), popup=node).add_to(mymap)
    folium.Marker(location=(loaded_graph.nodes[node]['y'],loaded_graph.nodes[node]['x']),popup=node).add_to(mymap)

# Plot edges on the map
for start,end,edge in loaded_graph.edges(data=True):
#     print(start,end,edge)
    start_node=start
    end_node=end
    if start_node==end_node or 'weight' not in edge:
        weight=0
    else:weight = edge['weight']
    folium.PolyLine([(loaded_graph.nodes[start_node]['y'],loaded_graph.nodes[start_node]['x']), (loaded_graph.nodes[end_node]['y'],loaded_graph.nodes[end_node]['x'])], color="black", weight=2.5, opacity=1, popup=f"{weight:.2f} meters").add_to(mymap)
# Plot the expanded nodes

for node in nodes_expanded:
    path_coordinates = [(loaded_graph.nodes[neighbour]['y'],loaded_graph.nodes[neighbour]['x'])for neighbour in loaded_graph.neighbors(node)]
    folium.PolyLine(path_coordinates, color="Yellow", weight=5, opacity=1).add_to(mymap)

# Plot the shortest path in red
distance=0
for i  in range(len(shortest_path)-1):
#     print(loaded_graph.get_edge_data(shortest_path[i],shortest_path[i+1])[0])
    distance+=loaded_graph.get_edge_data(shortest_path[i],shortest_path[i+1])[0]['weight']
path_coordinates = [(loaded_graph.nodes[node]['y'],loaded_graph.nodes[node]['x'])for node in shortest_path]
folium.PolyLine(path_coordinates, color="red", weight=5, opacity=1, popup=f"Shortest Path: {distance:.2f} meters").add_to(mymap)

# Save the map to an HTML file
mymap.save("map_with_nodes_Astar_octile.html")

A* with Chebyshev Heuristic

In [ ]:
#A* Search chebyshev
shortest_path,nodes_expanded=aStarSearch(loaded_graph,start_test,end_test,chebyshev_distance)

Plot A* path and nodes expanded with Chebyshev heuristic

In [ ]:
#Plot A* Map
# Create a folium map centered at the first node
mymap = folium.Map(location=list((loaded_graph.nodes[178537304]['y'],loaded_graph.nodes[178537304]['x'])), zoom_start=50)

# Plot each node on the map with its name
for node, coordinates in loaded_graph.nodes(data=True):
#     print(node,(loaded_graph.nodes[node]['y'],loaded_graph.nodes[node]['x']),coordinates['name'],coordinates)
#     folium.Marker(location=(loaded_graph.nodes[node]['y'],loaded_graph.nodes[node]['x']), popup=node).add_to(mymap)
    folium.Marker(location=(loaded_graph.nodes[node]['y'],loaded_graph.nodes[node]['x']),popup=node).add_to(mymap)

# Plot edges on the map
for start,end,edge in loaded_graph.edges(data=True):
#     print(start,end,edge)
    start_node=start
    end_node=end
    if start_node==end_node or 'weight' not in edge:
        weight=0
    else:weight = edge['weight']
    folium.PolyLine([(loaded_graph.nodes[start_node]['y'],loaded_graph.nodes[start_node]['x']), (loaded_graph.nodes[end_node]['y'],loaded_graph.nodes[end_node]['x'])], color="black", weight=2.5, opacity=1, popup=f"{weight:.2f} meters").add_to(mymap)
# Plot the expanded nodes

for node in nodes_expanded:
    path_coordinates = [(loaded_graph.nodes[neighbour]['y'],loaded_graph.nodes[neighbour]['x'])for neighbour in loaded_graph.neighbors(node)]
    folium.PolyLine(path_coordinates, color="Yellow", weight=5, opacity=1).add_to(mymap)

# Plot the shortest path in red
distance=0
for i  in range(len(shortest_path)-1):
#     print(loaded_graph.get_edge_data(shortest_path[i],shortest_path[i+1])[0])
    distance+=loaded_graph.get_edge_data(shortest_path[i],shortest_path[i+1])[0]['weight']
path_coordinates = [(loaded_graph.nodes[node]['y'],loaded_graph.nodes[node]['x'])for node in shortest_path]
folium.PolyLine(path_coordinates, color="red", weight=5, opacity=1, popup=f"Shortest Path: {distance:.2f} meters").add_to(mymap)

# Save the map to an HTML file
mymap.save("map_with_nodes_Astar_chebyshev.html")

Implementing Depth First Search

In [ ]:
#DFS
def depth_first_search_path(graph, start, end):
    stack = [(start, [start])]
    visited = set()

    while stack:
        (current, path) = stack.pop()
        visited.add(current)

        if current == end:
            return path, visited

        for neighbor in graph.neighbors(current):
            if neighbor not in visited:
                stack.append((neighbor, path + [neighbor]))
                visited.add(neighbor)

    return None, visited



In [ ]:
path, visited_nodes = depth_first_search_path(loaded_graph, start_test, end_test)

In [ ]:
print("No of Nodes Visited",len(visited_nodes))

Plot DFS on map

In [ ]:
import folium

# Create a folium map centered at the first node
mymap = folium.Map(location=list((loaded_graph.nodes[178537304]['y'],loaded_graph.nodes[178537304]['x'])), zoom_start=50)

# Plot each node on the map with its name
for node, coordinates in loaded_graph.nodes(data=True):
#     print(node,(loaded_graph.nodes[node]['y'],loaded_graph.nodes[node]['x']),coordinates['name'],coordinates)
#     folium.Marker(location=(loaded_graph.nodes[node]['y'],loaded_graph.nodes[node]['x']), popup=node).add_to(mymap)
    folium.Marker(location=(loaded_graph.nodes[node]['y'],loaded_graph.nodes[node]['x']),popup=node).add_to(mymap)

# Plot edges on the map
for start,end,edge in loaded_graph.edges(data=True):
#     print(start,end,edge)
    start_node=start
    end_node=end
    if start_node==end_node or 'weight' not in edge:
        weight=0
    else:weight = edge['weight']
    folium.PolyLine([(loaded_graph.nodes[start_node]['y'],loaded_graph.nodes[start_node]['x']), (loaded_graph.nodes[end_node]['y'],loaded_graph.nodes[end_node]['x'])], color="black", weight=2.5, opacity=1, popup=f"{weight:.2f} meters").add_to(mymap)

# Plot the expanded nodes
path_coordinates=[]
for node in visited_nodes:
    path_coordinates.append((loaded_graph.nodes[node]['y'],loaded_graph.nodes[node]['x']))
    folium.PolyLine(path_coordinates, color="Yellow", weight=5, opacity=1).add_to(mymap)

# Plot the shortest path in red
distance=0
for i  in range(len(path)-1):
#     print(loaded_graph.get_edge_data(shortest_path[i],shortest_path[i+1])[0])
    distance+=loaded_graph.get_edge_data(path[i],path[i+1])[0]['weight']
path_coordinates = [(loaded_graph.nodes[node]['y'],loaded_graph.nodes[node]['x'])for node in path]
folium.PolyLine(path_coordinates, color="red", weight=5, opacity=1, popup=f"Shortest Path: {distance:.2f} meters").add_to(mymap)

# Save the map to an HTML file
mymap.save("map_with_nodes_dfs.html")

Implementing Breath First Search

In [ ]:
def bfs(graph, start, end):
    visited = set()
    queue = deque([(start, [start])])
    visited_nodes = []  # To store visited nodes

    while queue:
        current_node, path = queue.popleft()
        visited.add(current_node)
        visited_nodes.append(current_node)

        if current_node == end:
            return path, visited_nodes

        for neighbor in graph.neighbors(current_node):
            if neighbor not in visited:
                queue.append((neighbor, path + [neighbor]))

    return None, visited_nodes

In [ ]:
path, visited_nodes = bfs(loaded_graph, start_test,end_test)

In [ ]:
print("No of Nodes Visited",len(visited_nodes))

Plot BFS on Map

In [ ]:
import folium

# Create a folium map centered at the first node
mymap = folium.Map(location=list((loaded_graph.nodes[178537304]['y'],loaded_graph.nodes[178537304]['x'])), zoom_start=30)

# Plot each node on the map with its name
for node, coordinates in loaded_graph.nodes(data=True):
#     print(node,(loaded_graph.nodes[node]['y'],loaded_graph.nodes[node]['x']),coordinates['name'],coordinates)
#     folium.Marker(location=(loaded_graph.nodes[node]['y'],loaded_graph.nodes[node]['x']), popup=node).add_to(mymap)
    folium.Marker(location=(loaded_graph.nodes[node]['y'],loaded_graph.nodes[node]['x']),popup=node).add_to(mymap)

# Plot edges on the map
for start,end,edge in loaded_graph.edges(data=True):
#     print(start,end,edge)
    start_node=start
    end_node=end
    if start_node==end_node or 'weight' not in edge:
        weight=0
    else:weight = edge['weight']
    folium.PolyLine([(loaded_graph.nodes[start_node]['y'],loaded_graph.nodes[start_node]['x']), (loaded_graph.nodes[end_node]['y'],loaded_graph.nodes[end_node]['x'])], color="black", weight=2.5, opacity=1, popup=f"{weight:.2f} meters").add_to(mymap)

# Plot the expanded nodes
path_coordinates=[]
for node in visited_nodes:
    path_coordinates.append((loaded_graph.nodes[node]['y'],loaded_graph.nodes[node]['x']))
    folium.PolyLine(path_coordinates, color="Yellow", weight=5, opacity=1).add_to(mymap)

# Plot the shortest path in red
distance=0
for i  in range(len(path)-1):
#     print(loaded_graph.get_edge_data(shortest_path[i],shortest_path[i+1])[0])
    distance+=loaded_graph.get_edge_data(path[i],path[i+1])[0]['weight']
path_coordinates = [(loaded_graph.nodes[node]['y'],loaded_graph.nodes[node]['x'])for node in path]
folium.PolyLine(path_coordinates, color="red", weight=5, opacity=1, popup=f"Shortest Path: {distance:.2f} meters").add_to(mymap)

# Save the map to an HTML file
mymap.save("map_with_nodes_bfs.html")